# Mortgage Lending Demo in Azure ML
This notebook trains a simple XGBoost classifier on your mortgage lending dataset, logs metrics and runs in Azure ML, and registers the resulting model.

## Prerequisites
- Upload `MORTGAGE_LENDING_DEMO_DATA.csv` into the **default datastore** of your Azure ML workspace (via Studio ▶ Data ▶ Datastores ▶ Browse ▶ Upload).
- Select the **Python 3.10 – AzureML** kernel on your Compute Instance.

In [ ]:
# Install the v1 SDK into this kernel
!pip install azureml-core --upgrade --quiet

In [ ]:
# Import necessary libraries
from azureml.core import Workspace, Experiment, Dataset, Model, Run
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib
import os
import pandas as pd

In [ ]:
# Connect to workspace and load data (replace with your actual values)
from azureml.core import Workspace

# 🔧 Replace the strings below with your own values
subscription_id  = "<YOUR_SUBSCRIPTION_ID>"
resource_group   = "<YOUR_RESOURCE_GROUP>"
workspace_name   = "<YOUR_WORKSPACE_NAME>"

ws = Workspace.get(
    name             = workspace_name,
    subscription_id  = subscription_id,
    resource_group   = resource_group
)
print(f"Workspace: {ws.name} (RG: {ws.resource_group})")


## Step 1: Read Model Training Data

In [ ]:
from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name="collegeaimlops")
df = ds.to_pandas_dataframe()
print(f"Loaded {len(df)} rows from Data asset")
df.head()

## Step 2: Prepare Model Features

In [ ]:
# Drop rows missing target or any required feature
required_cols = [
    "MORTGAGERESPONSE",        # <-- This is your target variable
    "APPLICANT_INCOME_000S",
    "LOAN_AMOUNT_000S",
    "LOAN_TYPE_NAME",          # <-- This is a categorical column
    "LOAN_PURPOSE_NAME",       # <-- This is a categorical column
    "COUNTY_NAME",             # <-- This is a categorical column
]
df_clean = df.dropna(subset=required_cols)

# One-hot encode the categoricals (drop_first to avoid dummy trap)
categorical_cols = ["LOAN_TYPE_NAME", "LOAN_PURPOSE_NAME", "COUNTY_NAME"]
df_encoded = pd.get_dummies(
    df_clean,
    columns=categorical_cols,
    drop_first=True
)

# Split out X & y, keeping only numeric columns
X = df_encoded.drop(columns=["MORTGAGERESPONSE"])
X = X.select_dtypes(include=["number"]).copy()

# Drop any rows that still have NaNs (just in case)
X = X.dropna()

y = df_encoded.loc[X.index, "MORTGAGERESPONSE"] # <-- Match this to your target variable

## Step 3: Split into Train, Validation, and Holdout Sets (Split 80/20 then 75/25)

In [ ]:
# First split: 80% temp (train + val), 20% holdout (0.2 is your first test_size)
X_temp, X_holdout, y_temp, y_holdout = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
# Second split: 75% train, 25% val (0.25 is your second test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Holdout: {len(X_holdout)}")

## Step 4: Train & Log XGBoost Model using 'logloss' as the evaluation metric in Azure ML

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

experiment = Experiment(workspace=ws, name="mortgage-xgb-demo")
run = experiment.start_logging()

model = XGBClassifier(eval_metric="logloss", use_label_encoder=False)
model.fit(X_train, y_train)

# 1) Get validation predictions
val_preds = model.predict(X_val)

# 2) Compute metrics
val_acc   = accuracy_score(y_val, val_preds)
precision = precision_score(y_val, val_preds, average="macro")
recall    = recall_score(y_val,    val_preds, average="macro")
f1        = f1_score(y_val,        val_preds, average="macro")

# 3) Log everything before closing the run
run.log("validation_accuracy", val_acc)
run.log("precision_macro",     precision)
run.log("recall_macro",        recall)
run.log("f1_macro",            f1)

print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Precision (macro):   {precision:.4f}")
print(f"Recall (macro):      {recall:.4f}")
print(f"F1 (macro):          {f1:.4f}")

run.complete()

## Step 5: Evaluate Model Performance on Validation Set

In [ ]:
# Step 5: Evaluate Model Performance on Validation Set (purely for display)
from sklearn.metrics import classification_report, confusion_matrix

# reuse val_preds from above
print("Classification Report:\n", classification_report(y_val, val_preds))
print("Confusion Matrix:\n", confusion_matrix(y_val, val_preds))

## Step 6: Allow Azure ML to Access Snowflake
In order for your Azure ML instance to connect to Snowflake, you must **add the current public IP address** of this notebook to your Snowflake network policy.

#### Step 6A: Get the Public IP of this Azure ML Instance

In [ ]:
!curl ifconfig.me

#### Step 6B: Add the IP to Your Snowflake Network Policy

Copy the Step 7A output (ex: `52.183.42.53`) and update your Snowflake network policy by running the following SQL in Snowsight (you must have `ACCOUNTADMIN` privileges):

##### Option 1: Create a new network policy (recommended if not already created)
```sql
-- Create it once
CREATE OR REPLACE NETWORK POLICY ALLOW_SAGEMAKER
  ALLOWED_IP_LIST = ('<YOUR_SAGEMAKER_IP>')
  COMMENT = 'Restrict access to SageMaker IPs for MLOps HOL';
-- Assign to service user only
ALTER USER mlops_user SET NETWORK_POLICY = ALLOW_SAGEMAKER;
-- Verify assignment
DESC USER mlops_user;
```
##### Option 2: Append to an existing IP allowlist (preserve existing IPs)
```sql
-- If you already have other IPs in the list, you can append your SageMaker IP like this:
ALTER NETWORK POLICY ALLOW_SAGEMAKER SET ALLOWED_IP_LIST = (
  'existing.ip.1',
  'existing.ip.2',
  '52.183.42.53'
);
```

## Step 7: Connect to Snowflake from Azure ML using a .toml File
This notebook uses a local .toml file to securely store your Snowflake connection parameters.

#### Step 7A: Create a connections.toml file
Create a connections.toml file locally with the following structure:
```
[connections.snowflake_conn]
account = "your_account"
user = "mlops_user"
role = "aicollege"
warehouse = "aicollege"
database = "aicollege"
schema = "public"
authenticator = "snowflake_jwt"
```
💡 Replace the values with your actual Snowflake connection details.

📝 Reminder: After updating the connections.toml file with your Snowflake account details, make sure to save the file before running the next step.
Otherwise, your notebook won’t be able to read the correct connection settings.

#### Step 7B: Upload the `.toml` and `.pem` Files to Azure ML

In Studio, go to Notebooks → click the Upload button in the file-browser pane.

Upload your connections.toml and your rsa_private_key.pem into the root directory (or a subfolder).

#### Step 7C: Validate Snowpark access with connections.toml

In [ ]:
import toml
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from snowflake.snowpark import Session

# 1) Load your connection settings
conn_cfg = toml.load("connections.toml")["connections"]["Snowpark_MLOps_HOL"]

# 2) Read your PEM and convert to DER bytes
with open("rsa_private_key.pem", "rb") as f:
    pem_data = f.read()

private_key = serialization.load_pem_private_key(
    pem_data,
    password=None,
    backend=default_backend()
)

der_bytes = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# 3) Build your Snowpark Session using DER‐encoded key
session = Session.builder.configs({
    "account":       conn_cfg["account"],
    "user":          conn_cfg["user"],
    "private_key":   der_bytes,
    "role":          conn_cfg["role"],
    "warehouse":     conn_cfg["warehouse"],
    "database":      conn_cfg["database"],
    "schema":        conn_cfg["schema"],
    "authenticator": conn_cfg["authenticator"],
}).create()

# 4) Test the connection
session.sql("SELECT current_user(), current_warehouse(), current_database(), current_schema()").show()

## Step 8: Register Your Azure ML Model in Snowflake Model Registry
> **Note:** Unlike AWS SageMaker’s built‐in estimator—which automatically serializes your trained model for you—Azure ML requires you to explicitly save (e.g., via `joblib.dump`) your model artifact before registration.

1. **Ensure your Snowflake session is initialized**  
   You should already have a working `snowflake.snowpark.Session` in your notebook.

2. **Locate your model artifact**  
   If you followed the earlier steps, your XGBoost model was saved at:
   ```python
   model_path = "outputs/mortgage_xgb_model.pkl"
   ```
3. **Call `log_model()`**
   Import and invoke the Snowflake ML API to register the model:
   ```python 
   from snowflake.ml.model import log_model

   log_model(
   model_export_path=model_path,
   model_name="mortgage_xgb_model",
   model_type="xgboost",
   registration_database="YOUR_DB",
   registration_schema="YOUR_SCHEMA",
   registration_warehouse="YOUR_WAREHOUSE",
   replace=True        # overwrite existing version if present
   )
   ```
4. **What happens under the hood?**
   - **Manual serialization** of your model file into DER/Pickle.
   - **Automatic versioning** and **metadata capture** (framework, tags, metrics).
   - **No extra staging**—the API handles uploading for you.
5. Why use Snowflake’s Model Registry?
   - **Unified storage** beside your data.
   - **RBAC & governance** at the schema level.
   - **SQL + Python inference** on any table with compatible features.
   - **Lineage & observability** for tracking performance and drift.

💡 **Pro Tip:** To pull an Azure ML–registered model first, you can use:
```python
from azureml.core.model import Model
model_path = Model(ws, "mortgage-xgb-model").download(target_dir=".")
```
then pass that `model_path` into `log_model()` as shown above.

#### Step 8A: Create Small Input Data for Snowflake Model Registry
It helps Snowflake infer the input schema so it knows how to call the model later (during inference, for example). 

Using a small sample is a best practice to keep things fast and lightweight.

In [ ]:
# Take a tiny float32 sample of the features you actually trained on
sample_input_data = (
    X_train
    .astype("float32")
    .sample(5, random_state=42)
)

#### Step 8B: Log the trained Azure ML model to Snowflake Model Registry

In [ ]:
from snowflake.ml.registry import Registry
from snowflake.ml.model import type_hints

# Grab the metrics you logged in Azure ML
metrics = run.get_metrics()

# Initialize the Registry pointing at your target database & schema
reg = Registry(
    session       = session,
    database_name = conn_cfg["database"],
    schema_name   = conn_cfg["schema"]
)

# Register the model object you trained in Azure ML
mv = reg.log_model(
    model_name        = "azureml_xgb_model",                        # provide required MLOPs HOL model name
    version_name      = "v1",                                       # provide required version name
    model             = model,                                      # your XGBClassifier instance
    sample_input_data = sample_input_data,                          # small pandas df for signature
    metrics           = metrics,                                    # dict from run.get_metrics()
    conda_dependencies= ['xgboost'],                                # your model’s runtime deps
    comment           = "XGBoost classifier trained in Azure ML",   # optional description
    task              = type_hints.Task.TABULAR_BINARY_CLASSIFICATION,
    options           = {'relax_version': False}                    # suppress reproducibility warning
)

print(f"✅ Model registered as {mv.model_name} version {mv.version_name}")

#### Step 8C: Create and Apply Tags for Model Versioning

In [ ]:
# Create or replace your governance tags in Snowflake
session.sql("CREATE OR REPLACE TAG MODEL_STAGE_TAG").collect()
session.sql("CREATE OR REPLACE TAG MODEL_PURPOSE_TAG").collect()
session.sql("CREATE OR REPLACE TAG SOURCE").collect()
session.sql("CREATE OR REPLACE TAG PROJECT").collect()

In [ ]:
# Apply tags and model-level metadata
m = reg.get_model("azureml_xgb_model")   # same name you used when registering

# Add model-level description
m.comment = "XGBoost classifier trained in Azure ML to predict mortgage approval."

# Attach your predefined tags for governance
m.set_tag("MODEL_STAGE_TAG",   "PROD")                               # deployment stage
m.set_tag("MODEL_PURPOSE_TAG", "Mortgage Response Classification")   # business context
m.set_tag("SOURCE",            "Azure ML")                           # origin
m.set_tag("PROJECT",           "College of AI - MLOps HOL")          # for traceability

# View the tags you’ve just set
m.show_tags()

## Step 9: Preprocess Inference Data for Batch Scoring
Before we run batch inference, we need to make sure the incoming Snowflake data matches the **feature format the model was trained on**.

Our Azure Machine Learning model expects:
- All numeric features
- One-hot encoded columns for `LOAN_TYPE_NAME`, `LOAN_PURPOSE_NAME`, and `COUNTY_NAME`  
- The same column names (and order) as the training data

In this step, we:
1. Load `InferenceMortgageData` from Snowflake  
2. Apply one-hot encoding with Pandas  
3. Fill in any missing dummy columns with 0  
4. Reorder columns to match `X_train`

In [ ]:
# Pull your inference table into a Pandas DataFrame
inference_df = (
    session
    .table("INFERENCEMORTGAGEDATA")     # or session.sql("SELECT * FROM INFERENCEMORTGAGEDATA")
    .filter("WEEK = 1")                 # only get data associated with WEEK = 1
    .to_pandas()                        # use .to_pandas() in Snowpark
)

# 2) One-hot encode the same categoricals
inference_encoded = pd.get_dummies(
    inference_df,
    columns=categorical_cols,  # reuse the variable you defined in Step 2
    drop_first=True
)

# 3) Add any missing dummy columns (that X_train had) and fill with 0
for col in X_train.columns:
    if col not in inference_encoded:
        inference_encoded[col] = 0

# 4) Reorder to match the training feature order
inference_features = inference_encoded[X_train.columns]

# Now inference_features is ready to feed into your model:
preds = mv.run(
    inference_features,
    function_name="predict"
)

## Step 10: Batch-scoring Examples

In real projects you’ll pick the approach that best matches your scale, latency, and operational constraints. We illustrate three patterns:

1. **Step 10A: Quick Demo (Week 1 only)**  
   A tiny slice of your data scored via Pandas → Snowflake.  
   — ✅ Instant feedback  
   — ✅ Perfect for interactive exploration  

2. **Step 10B: Full-Dataset Scoring (Pandas-first)**  
   Pull all weeks into a Pandas DataFrame, preprocess & align features as during training, run inference with your Snowflake–registered model, then write back one big table.  
   — ✅ Familiar end-to-end Python  
   — ⚠️ Watch notebook memory on large tables  

***Appendix: Snowflake-Native Scoring***  
Push your one-hot & alignment logic into a VIEW and call `mv.run()` on that Snowpark DataFrame. This removes the Pandas round-trip but invokes Python UDFs under the hood, which can be slower at scale.

#### Step 10A: Week 1 Mortgage Application Scoring
Mortgage application data is continuously collected and stored in Snowflake. Rather than scoring in real-time, we use **batch inference** to evaluate applications in bulk — a common approach when scoring latency is not critical.

In this step you will:
1. **Run inference** on Week 1 application data using your registered XGBoost model (specifying the `predict` entry point).  
2. **Format and rename the prediction output** for clarity.  
3. **Join predictions with the true outcome** (`MORTGAGERESPONSE`) for monitoring.  
4. **Save the results** into a unified table (`PREDICTIONS_WITH_GROUND_TRUTH`) for downstream analysis and drift detection.

This unified table enables model observability by tracking how predictions align with actual outcomes over time.

> **Reminder:** Your model expects the exact same one-hot-encoded, numeric-only schema you built in Step 11 (`inference_features`).

In [ ]:
## Run Batch Inference on Week 1 Data and Save Unified Output Table

import numpy as np
import pandas as pd
from IPython.display import display

# You already have:
# - `session`: your Snowpark Session
# - `inference_df`: raw WEEK 1 data as pandas
# - `inference_features`: one-hot encoded & aligned features as pandas
# - `mv`: your ModelVersion object from the Snowflake Registry

# Run inference on your preprocessed features
predictions      = mv.run(inference_features,   function_name="predict")
proba_predictions = mv.run(inference_features,  function_name="predict_proba")

# Convert outputs to pandas Series
pred_series  = pd.Series(np.squeeze(predictions),      name="PREDICTION")
score_series = pd.Series(np.array(proba_predictions)[:,1], name="PREDICTED_SCORE")

# Merge predictions back onto the raw input for observability
results_df = inference_df.copy()
results_df["WEEK"]            = 1
results_df["PREDICTION"]      = pred_series
results_df["PREDICTED_SCORE"] = score_series

# Write the unified results back into Snowflake
session.write_pandas(
    results_df,
    table_name="PREDICTIONS_WITH_GROUND_TRUTH",
    auto_create_table=True,
    overwrite=True
)

# Pull the Snowflake table into pandas
pdf = (
    session
    .table("PREDICTIONS_WITH_GROUND_TRUTH")
    .to_pandas()
)
display(pdf)

#### Step 10B: Full-Table Scoring (All Weeks) — Pandas-backed

Now that you’ve validated scoring on a tiny slice, let’s pull the **entire** mortgage application table into **Pandas** and score it in one shot:

- **Load all weeks** into a Pandas DataFrame using `session.table("INFERENCEMORTGAGEDATA").to_pandas()`.  
- **One-hot encode & align** the same categorical columns in Pandas so your DataFrame matches `X_train` exactly.  
- **Run inference** via `mv.run(..., function_name="predict")` and `mv.run(..., function_name="predict_proba")` on the Pandas DataFrame.  
- **Merge predictions** back onto your raw data and write the full results into Snowflake with `session.write_pandas(..., overwrite=True)`.

> **Tip:** This end-to-end Pandas approach feels familiar and scales well up to hundreds of thousands (even low millions) of rows—just keep an eye on your notebook’s memory if the table grows very large.

In [ ]:
import pandas as pd
import numpy as np
from snowflake.ml.registry import Registry

# 1) Load the _entire_ inference dataset into pandas
raw_all = session.table("INFERENCEMORTGAGEDATA").to_pandas()

# 2) One-hot encode the same categoricals (drop_first to match training)
encoded_all = pd.get_dummies(
    raw_all,
    columns=categorical_cols,
    drop_first=True
)

# 3) Add any columns X_train had that this week didn’t, then reorder
for c in X_train.columns:
    if c not in encoded_all.columns:
        encoded_all[c] = 0
encoded_all = encoded_all[X_train.columns]

# 4) Load your model version from the registry
reg = Registry(session=session,
               database_name=conn_cfg["database"],
               schema_name=conn_cfg["schema"])
model = reg.get_model("azureml_xgb_model")    # <-- match your model_name

# 5) Run inference
preds  = mv.run(encoded_all,     function_name="predict")
probas = mv.run(encoded_all,     function_name="predict_proba")

# 6) Attach back into the raw dataframe
raw_all["PREDICTED_RESPONSE"] = np.squeeze(preds)
raw_all["PREDICTED_SCORE"]    = np.array(probas)[:,1]

# 7) Push it all back into Snowflake
results_sp = session.create_dataframe(raw_all)
results_sp.write.mode("overwrite") \
          .save_as_table("ALL_PREDICTIONS_WITH_GROUND_TRUTH")

# 8) Quick sanity-check
pdf = results_sp.limit(10).to_pandas()
display(pdf)

## Scaling Inference with Snowflake ML Jobs and SPCS

Once you've validated your **Pandas-backed** batch scoring approach, production workloads typically require more scalable execution environments. Snowflake provides two powerful options:

### 📋 **Snowflake ML Jobs**
Schedule and orchestrate your end-to-end ML pipelines using Snowflake's managed **Container Runtime**.

**Key Benefits:**
-✅ **IDE Integration**: Dispatch jobs from VS Code, PyCharm, or Azure ML
- ✅ **Serverless Execution**: Auto-scaling with no infrastructure management  
- ✅ **Cost-Effective**: Pay only for compute time used

**Best For:** Scheduled batch inference, model retraining pipelines, feature engineering workflows

---

### 🐳 **Snowpark Container Services (SPCS)**
Deploy your model as containerized services or jobs on dedicated compute pools with full flexibility.

**Key Benefits:**
- ✅ **No Package Restrictions**: Use any Python packages from PyPI
- ✅ **GPU Support**: Scale to large models with distributed GPU clusters
- ✅ **Service Endpoints**: Deploy always-on inference APIs

**Best For:** Real-time inference endpoints, large-scale bulk scoring, GPU-accelerated workloads

---

### 🎯 **Choosing the Right Approach**

| Scenario | Recommended Solution | Key Consideration |
|----------|---------------------|-------------------|
| **Scheduled batch inference** | ML Jobs | Finite-duration, cost-effective |
| **Always-on inference API** | SPCS Services | Set `MIN_INSTANCES = MAX_INSTANCES` |
| **Large bulk scoring** | SPCS Jobs | Handle enterprise-scale datasets |

### 📚 **Documentation & Next Steps**

- **[ML Jobs Guide](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-mlops)** - Automated ML pipelines
- **[SPCS Overview](https://docs.snowflake.com/en/developer-guide/snowpark-container-services/overview)** - Container services
- **[Model Serving](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/model-serving-spcs)** - Deploy models to SPCS

**💡 Pro Tip:** Start with ML Jobs for your weekly batch scoring, then consider SPCS if you need real-time inference or GPU acceleration.

---